# Dataset

HMDA mortgage application data for 2018

Link: https://ffiec.cfpb.gov/data-publication/snapshot-national-loan-level-dataset/2018

File: Loan/Application Records (LAR)

Data Glossary: https://ffiec.cfpb.gov/documentation/2018/lar-data-fields/


# Problem
Posed as a binary classification problem to predict whether mortgage is approved or not

In [1]:
# Import required packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

# File Read

In [2]:
# Read data 
data = pd.read_csv('2018_public_lar_csv.csv')

In [3]:
print('Total number of data points - ', len(data))

Total number of data points -  4007401


In [4]:
# Select only male and female applicants.
# Select only application with status 'application denied' and 'purchase loan'
g_data = data.loc[(data['applicant_sex'].isin([1,2])) & (data['action_taken'].isin([3,6]))]

In [5]:
print('Number of filtered data - ', len(g_data))

Number of filtered data -  661601


In [6]:
# Print first few rows
g_data.head()

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
16,2018,549300HW662MN1WU8550,35084,NJ,34013,3.4013e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3051.0,99.44,100700.0,60.0,329.0,1048.0,71.0
31,2018,549300HW662MN1WU8550,23104,TX,48439,4.84391e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,8848.0,47.74,75000.0,105.0,2043.0,2488.0,20.0
34,2018,549300HW662MN1WU8550,38060,AZ,4013,4.01312e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,9.0,NaN,NaN,4674.0,86.52,69100.0,63.0,701.0,1257.0,28.0
39,2018,549300HW662MN1WU8550,45300,FL,12101,1.2101e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,4678.0,23.19,63900.0,52.0,974.0,1798.0,36.0
45,2018,549300HW662MN1WU8550,29460,FL,12105,1.2105e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,NaN,NaN,NaN,2180.0,5.83,53600.0,142.0,718.0,1083.0,53.0


In [7]:
# For the sake of experiment select only first 20,000 rows
input_data = g_data[0:20000]

In [8]:
# Print first few rows 
input_data.head()

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
16,2018,549300HW662MN1WU8550,35084,NJ,34013,3.4013e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3051.0,99.44,100700.0,60.0,329.0,1048.0,71.0
31,2018,549300HW662MN1WU8550,23104,TX,48439,4.84391e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,8848.0,47.74,75000.0,105.0,2043.0,2488.0,20.0
34,2018,549300HW662MN1WU8550,38060,AZ,4013,4.01312e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,9.0,NaN,NaN,4674.0,86.52,69100.0,63.0,701.0,1257.0,28.0
39,2018,549300HW662MN1WU8550,45300,FL,12101,1.2101e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,4678.0,23.19,63900.0,52.0,974.0,1798.0,36.0
45,2018,549300HW662MN1WU8550,29460,FL,12105,1.2105e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,NaN,NaN,NaN,2180.0,5.83,53600.0,142.0,718.0,1083.0,53.0


# Select data 

In [9]:
# Select only the below columns
selected_data  = input_data[['conforming_loan_limit','preapproval','loan_term', 'aus_1', 
                             'applicant_age', 'applicant_credit_score_type', 'combined_loan_to_value_ratio', 
                             'property_value', 'income', 'debt_to_income_ratio','loan_amount', 
                             'applicant_ethnicity_1', 'applicant_race_1', 'applicant_sex', 'action_taken']]

In [10]:
# Display only first few rows
selected_data.head()

,conforming_loan_limit,preapproval,loan_term,aus_1,applicant_age,applicant_credit_score_type,combined_loan_to_value_ratio,property_value,income,debt_to_income_ratio,loan_amount,applicant_ethnicity_1,applicant_race_1,applicant_sex,action_taken
16,C,2.0,360,1.0,25-34,3.0,98.25,285000,74.0,40,285000.0,2.0,5.0,1.0,3.0
31,C,2.0,360,1.0,25-34,1.0,94,245000,50.0,>60%,235000.0,2.0,2.0,2.0,3.0
34,C,2.0,360,2.0,25-34,1.0,111.06,135000,35.0,37,155000.0,1.0,5.0,2.0,3.0
39,C,2.0,360,1.0,25-34,2.0,111.64,105000,31.0,37,115000.0,3.0,6.0,1.0,3.0
45,C,2.0,360,1.0,25-34,1.0,96.5,155000,34.0,>60%,155000.0,1.0,5.0,2.0,3.0


In [11]:
# Check if there is any null values
selected_data.isna().sum()

conforming_loan_limit             19
preapproval                        0
loan_term                         22
aus_1                              0
applicant_age                      0
applicant_credit_score_type        0
combined_loan_to_value_ratio    4428
property_value                   121
income                            83
debt_to_income_ratio            4557
loan_amount                        0
applicant_ethnicity_1              0
applicant_race_1                   0
applicant_sex                      0
action_taken                       0
dtype: int64

In [12]:
for col in selected_data.columns:
    if col in ['income', 'loan_amount', 'property_value', 'combined_loan_to_value_ratio']:
        selected_data.loc[(selected_data[col] == 'Exempt', col)] = 0000
        selected_data[col] = pd.to_numeric(selected_data[col])
        continue
    print(col)
    selected_data.loc[(selected_data[col] == 'Exempt', col)] = 0000
    print(selected_data[col].unique())
    print('--------------------------------')

conforming_loan_limit
['C' 'NC' nan 'U']
--------------------------------
preapproval
[2.]
--------------------------------
loan_term
[360 180 300 240 336 168 120 288 nan '360' '240' '180' '120' '300' 0 '168'
 '288' 252 324 12 216 348.0 144 156 312 84.0 60.0 372.0 192.0 186.0 187.0
 309.0 183.0 303.0 198.0 126.0 184.0 66.0 318.0 306.0 123.0 371.0 18.0
 24.0 6.0 14.0 '12' '6' '18']
--------------------------------
aus_1
[1.000e+00 2.000e+00 4.000e+00 6.000e+00 3.000e+00 1.111e+03 5.000e+00]
--------------------------------
applicant_age
['25-34' '45-54' '<25' '65-74' '35-44' '55-64' '>74' '8888']
--------------------------------
applicant_credit_score_type
[3.000e+00 1.000e+00 2.000e+00 9.000e+00 1.111e+03 7.000e+00 5.000e+00
 8.000e+00]
--------------------------------
debt_to_income_ratio
['40' '>60%' '37' '45' '50%-60%' '47' '41' '20%-<30%' '<20%' '49' '42'
 '30%-<36%' '38' '44' '46' '36' '43' '39' '48' nan 0]
--------------------------------
applicant_ethnicity_1
[ 2.  1.  3. 14. 11

In [13]:
selected_data.fillna(0, inplace=True)

In [14]:
selected_data.isna().sum()

conforming_loan_limit           0
preapproval                     0
loan_term                       0
aus_1                           0
applicant_age                   0
applicant_credit_score_type     0
combined_loan_to_value_ratio    0
property_value                  0
income                          0
debt_to_income_ratio            0
loan_amount                     0
applicant_ethnicity_1           0
applicant_race_1                0
applicant_sex                   0
action_taken                    0
dtype: int64

In [15]:
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 16 to 138314
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   conforming_loan_limit         20000 non-null  object 
 1   preapproval                   20000 non-null  float64
 2   loan_term                     20000 non-null  object 
 3   aus_1                         20000 non-null  float64
 4   applicant_age                 20000 non-null  object 
 5   applicant_credit_score_type   20000 non-null  float64
 6   combined_loan_to_value_ratio  20000 non-null  float64
 7   property_value                20000 non-null  float64
 8   income                        20000 non-null  float64
 9   debt_to_income_ratio          20000 non-null  object 
 10  loan_amount                   20000 non-null  float64
 11  applicant_ethnicity_1         20000 non-null  float64
 12  applicant_race_1              20000 non-null  float64
 13 

In [16]:
# Select only categorical columns
categorical_columns = ['conforming_loan_limit', 'preapproval', 'loan_term', 'aus_1', 'applicant_age', 
                      'applicant_credit_score_type', 'debt_to_income_ratio', 'applicant_ethnicity_1',
                      'applicant_race_1',  'applicant_sex', 'action_taken']

In [17]:
#Deep copy the original data
data_encoded = selected_data.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical_columns:
        print(col)
        if selected_data[col].dtype != object:
            data_encoded[col] = lab_enc.fit_transform(selected_data[col])
        else:
            data_encoded[col] = lab_enc.fit_transform(selected_data[col].astype(str))
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)

conforming_loan_limit
Feature conforming_loan_limit
mapping {'0': 0, 'C': 1, 'NC': 2, 'U': 3}
preapproval
Feature preapproval
mapping {2.0: 0}
loan_term
Feature loan_term
mapping {'0': 0, '12': 1, '12.0': 2, '120': 3, '120.0': 4, '123.0': 5, '126.0': 6, '14.0': 7, '144': 8, '144.0': 9, '156': 10, '168': 11, '18': 12, '18.0': 13, '180': 14, '180.0': 15, '183.0': 16, '184.0': 17, '186.0': 18, '187.0': 19, '192.0': 20, '198.0': 21, '216': 22, '24.0': 23, '240': 24, '240.0': 25, '252': 26, '288': 27, '300': 28, '300.0': 29, '303.0': 30, '306.0': 31, '309.0': 32, '312': 33, '312.0': 34, '318.0': 35, '324': 36, '336': 37, '348.0': 38, '360': 39, '360.0': 40, '371.0': 41, '372.0': 42, '6': 43, '6.0': 44, '60.0': 45, '66.0': 46, '84.0': 47}
aus_1
Feature aus_1
mapping {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 5, 1111.0: 6}
applicant_age
Feature applicant_age
mapping {'25-34': 0, '35-44': 1, '45-54': 2, '55-64': 3, '65-74': 4, '8888': 5, '<25': 6, '>74': 7}
applicant_credit_score_type
Featu

In [19]:
data_encoded.to_csv('preprocessed_data.csv', index=False)